# Block Creation based on Block Size 

Overall description coming...

## Imports

In [1]:
# For paths
import os 

# For data processing
import torch
import numpy as np
from torch.utils.data import Dataset
import math


# For 3D visualization
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

## Functions and variables to be used in this notebook

In [ ]:
# Setting the root for this repo
ROOT = os.path.realpath("..")

# Creating/ Making the point cloud
def makePC(point_data, color_data=np.array([])):
    pcd = o3d.geometry.PointCloud() #Create PC object
    pcd.points = o3d.utility.Vector3dVector(point_data) #Give coordinates
    #Coloring the PC
    if len(color_data) == 0:
        pcd.paint_uniform_color([1, 0, 0])
    else:
        pcd.colors = o3d.utility.Vector3dVector(color_data)
    return pcd

## Loading of the input data

Before starting processing ot doing anything significant we will have to load in the input data to start investigating and testing. After the loading in of the data we print out certain characteristics of the respective sample.

In [ ]:
# Path creation for easier loading
input_pc_path = ROOT + "/data/testdata/data_labelled_int.npy"

# Loading relevant data
input_pc = np.load(input_pc_path) #ndarray
input_pc_points, input_pc_labels = input_pc[:, 0:6], input_pc[:, 6]

# Print checks for the input data
print("--------- Print checks for input data ---------")
print(f"The input data consists of {len(input_pc)} points.")
print(f"The input data is of shape: {input_pc.shape}")
print(f"The input data contains {np.count_nonzero(input_pc_labels)} fracture points.")
print(f"Therefore, {len(input_pc) - np.count_nonzero(input_pc_labels)} points are non-fracture points in the entire sample.")
print(f"This means that {np.round(np.count_nonzero(input_pc_labels) / len(input_pc) * 100, 4)}% of all points are fracture points")
print("------------------------------------------------")

## Block center coordinate creation based on block size parameter

In the example above the block size is not really used for the block and center coordinate creation, therefore it will be adjusted according to the block size, while keeping the half-block-size padding.

In [ ]:
# Given HP and parameters
num_point = 4096
block_size = 20.0
sample_rate = 1.0

# Determining maximum and minimum points
coord_min, coord_max = np.amin(input_pc_points, axis=0)[:3], np.max(input_pc_points, axis=0)[:3]

# Determining maximum block resolution
max_blocks_x_res = math.ceil(coord_max[0] / block_size) 
max_blocks_y_res = math.ceil(coord_max[1] / block_size)
# Actual resolution after half block padding on each side
blocks_x_res = max_blocks_x_res - 1
blocks_y_res = max_blocks_y_res - 1
num_blocks = blocks_x_res * blocks_y_res
blocks_center_x_start = coord_min[0] + block_size
blocks_center_y_start = coord_min[1] + block_size
#block_step = block_size

# Center coordinate creation
blocks_center_x_coords = np.linspace(start=blocks_center_x_start, 
                                     stop=coord_max[0]-block_size, 
                                     num=blocks_x_res)
blocks_center_y_coords = np.linspace(start=blocks_center_y_start, 
                                     stop=coord_max[1]-block_size, 
                                     num=blocks_y_res)
blocks_center_xx_coords, blocks_center_yy_coords = np.meshgrid(blocks_center_x_coords, 
                                                               blocks_center_y_coords)
blocks_center_zz_coords = np.zeros((blocks_center_xx_coords.shape))
blocks_center_coords = np.hstack([blocks_center_xx_coords.reshape((-1,1)), 
                                  blocks_center_yy_coords.reshape((-1,1)), 
                                  blocks_center_zz_coords.reshape((-1,1))])

Now we have created the block center coordinates based on the block size parameter for this model. As a next step we now have to (double-)check whether, if blocked like this we might be able to get even nearly as many points as we need per block.

## Analysis of block creation based on block size

Coming...

In [ ]:
# Placeholder variables to fill with values 
block_point_idxs = [] #actual indices of points of input pc within respective block
block_num_frac_points = [] #actual labels of points of input pc within respective block
block_num_input_pc_points = [] #number of points of input pc in respective block
block_mins = [] #block minimum coordinate
block_maxs = [] #block maximum coordiante


# Looping through the created block centers to check for number of points
for index, block_center in enumerate(blocks_center_coords):
    block_min = block_center[:2] - [block_size / 2.0, block_size / 2.0]
    block_max = block_center[:2] + [block_size / 2.0, block_size / 2.0]
    # Determining the number of points in the block
    points_in_block = np.where((input_pc_points[:, 0] >= block_min[0]) & \
                               (input_pc_points[:, 0] <= block_max[0]) & \
                               (input_pc_points[:, 1] >= block_min[1]) & \
                               (input_pc_points[:, 1] <= block_max[1]))[0]
    #Pulling corresponding values for each block for analysis
    selected_points_idxs = input_pc_points[points_in_block]
    block_point_idxs.append(selected_points_idxs)
    selected_points_labels = input_pc_labels[points_in_block]
    block_num_frac_points.append(np.sum(selected_points_labels))
    block_num_input_pc_points.append(points_in_block.size)
    block_mins.append(block_min)
    block_maxs.append(block_max)


# Preparation for the print checks of the input pc
points_in_blocks_with_enough_points = [i for i in block_num_input_pc_points if i >= num_point]
idxs_blocks_with_enough_points = np.array([idx for idx, val in enumerate(block_num_input_pc_points) if val >= num_point], dtype=int)
points_in_blocks_with_fewer_points = [i for i in block_num_input_pc_points if i < num_point]
idxs_blocks_with_fewer_points = np.array([idx for idx, val in enumerate(block_num_input_pc_points) if val < num_point], dtype=int)
num_blocks_without_any_points = [i for i in block_num_input_pc_points if i == 0]
indices_of_blocks_with_zero_points = [idx for idx, val in enumerate(block_num_input_pc_points) if val == 0]
num_blocks_with_too_little_points = num_blocks - len(points_in_blocks_with_enough_points)
general_block_frac_point_avg = np.round(np.mean(block_num_frac_points), 2)
blocks_with_enough_points_frac_point_avg = np.round(np.mean(np.array(block_num_frac_points)[idxs_blocks_with_enough_points]), 2)
blocks_with_fewer_points_frac_point_avg = np.round(np.mean(np.array(block_num_frac_points)[idxs_blocks_with_fewer_points]), 2)


# Print checks for the blocks within the input pc
print('---------- Print checks for the input pc blocks ----------')
print(f'The input point cloud was divided into {len(blocks_center_coords)} blocks.')
print(f'The number of blocks that have more points than the subsampling goal: {len(points_in_blocks_with_enough_points)}')
print(f'The number of blocks that have less points then the subsampling goal: {num_blocks_with_too_little_points}')
print(f'The number of blocks which have no subsampled points at all: {len(indices_of_blocks_with_zero_points)}')
print(f'The maximum amount of points within a block is: {np.max(points_in_blocks_with_enough_points)}')
print(f'The average amount of points of the blocks that have enough points is: {np.round(np.mean(points_in_blocks_with_enough_points), 2)}')
print(f'The average amount of points of the blocks that have to few points is: {np.round(np.mean(points_in_blocks_with_fewer_points), 2)}')
print(f'On average there are {general_block_frac_point_avg} fracture points within a block.')
print(f'On average the blocks with enough points have {blocks_with_enough_points_frac_point_avg} fracture points within a block.')
print(f'On average the blocks with fewer points have {blocks_with_fewer_points_frac_point_avg} fracture points within a block.')
print(f'The amount of blocks that do not contain any fracture points are: {block_num_frac_points.count(0)}')

After this investigation with the parameters set in this fashion we end up with almost roughly half the amount of blocks having enough points (80), but essentially a lot more points on average (8414), while the average number of points in blocks with too few points seems too low to let it go into the model (645). Besides the amount of fracture points also differs greatly between these two groups and as well. Furthermore, with the parameters set like this there seem to be 60 blocks, that do not even showcase any fracture points, while 40 blocks do not even contain any points. As this sounds very intriguing but almost not possible, we will check this below to make sure it is correct. 

In [2]:
#TODO: Analyze the blocks with no points at all and also no fracture points!